In [1]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import ec

# Generate ECC private key
def generate_ecc_private_key():
    private_key = ec.generate_private_key(ec.SECP192R1(), default_backend())
    return private_key

# Generate ECDH shared key
def generate_ecdh_shared_key(private_key, peer_public_key_bytes):
    peer_public_key = serialization.load_pem_public_key(peer_public_key_bytes, backend=default_backend())
    shared_key = private_key.exchange(ec.ECDH(), peer_public_key)
    return shared_key

# Example usage
# Alice generates her private key
# alice_private_key = generate_ecc_private_key()
# Bob generates his private key
serverPriKey = generate_ecc_private_key()

# # Alice sends her public key to Bob
# alice_public_key_bytes = alice_private_key.public_key().public_bytes(
#     encoding=serialization.Encoding.PEM,
#     format=serialization.PublicFormat.SubjectPublicKeyInfo
# )

# Bob sends his public key to Alice
serverPubKeyBytes = serverPriKey.public_key().public_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PublicFormat.SubjectPublicKeyInfo
)

# Alice computes the shared key using Bob's public key
# alice_shared_key = generate_ecdh_shared_key(alice_private_key, bob_public_key_bytes)

# Bob computes the shared key using Alice's public key
# bob_shared_key = generate_ecdh_shared_key(bob_private_key, alice_public_key_bytes)

# The shared keys should match
# assert alice_shared_key == bob_shared_key
# print("Shared Key:", alice_shared_key.hex())
# def createNewKey():
#     global serverPriKey, serverPubKeyBytes
#     serverPriKey = generate_ecc_private_key()
#     serverPubKeyBytes = clientPrivateKey.public_key().public_bytes(
#     encoding=serialization.Encoding.PEM,
#     format=serialization.PublicFormat.SubjectPublicKeyInfo)
#     global 

randomNo = ''
clientPubKey = ''

In [2]:
Cryptkey = b'\xb9t5RR\x9d\x1d+\xaa\xa3\xd1}\xfb\x1a\x1d\xcb'
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad, unpad

def encrypt_AES(Cryptkey, data):
    data = bytes.fromhex(data)
    cipher = AES.new(Cryptkey, AES.MODE_CBC)
    ciphertext = cipher.encrypt(pad(data, AES.block_size))
    iv = cipher.iv
    return iv + ciphertext

def decrypt_AES(Cryptkey, encrypted_data):
    encrypted_data = bytes.fromhex(encrypted_data)
    iv = encrypted_data[:AES.block_size]
    ciphertext = encrypted_data[AES.block_size:]
    cipher = AES.new(Cryptkey, AES.MODE_CBC, iv)
    plaintext = unpad(cipher.decrypt(ciphertext), AES.block_size)
    return plaintext

In [3]:
import numpy as np
def SeedGenerator():
    return get_random_bytes(4).hex()
# SeedGenerator()
# import os
# print(os.urandom(1).hex())
def ValLevel(lvl):
    if len(lvl)==1:
        return '0'+lvl
    else:
        return lvl

In [4]:
def SeedGen(seed):
    global Cryptkey
    print(type(seed))
    return decrypt_AES(Cryptkey,seed).hex()
def ValidateRequest(from_client, step,key=''):
    secAccLvl = int(from_client[3:5])
    global randomNo, clientPubKey
    if from_client[0:2] == '27' and secAccLvl%2==1:
        if step == 0:
            clientPubKey = bytes.fromhex(from_client[6:len(from_client)])
            randomNo = generate_ecdh_shared_key(serverPriKey,clientPubKey)
            seed = SeedGenerator()
            temp = '67 ' + ValLevel(str(secAccLvl)) + ' '+ seed + ' '+ serverPubKeyBytes.hex()
            print(temp)
            return [temp, seed]
        else : 
            print("Wrong request")
            return ['End','']
    if from_client[0:2] == '27' and secAccLvl%2==0 and key!='':
        if step == 1 and randomNo.hex()+key == SeedGen(from_client[6:len(from_client)]):
            print('67 '+ ValLevel(str(secAccLvl)),'')
            return ['67 '+ ValLevel(str(secAccLvl)),'']        
        else : 
            print("Wrong request/key not matched/empty")
            return ['End','']
    

In [50]:
import socket
import pickle
serv = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
serv.bind(('0.0.0.0', 8080))
serv.listen(5)
a=0
b =0


while a==0:
  conn, addr = serv.accept()
  from_client = ''
  step = 0
  while b==0:
    data = conn.recv(4096)
    if not data: break
    from_client = data.decode('utf8')
    print (from_client)
    msg = 'ECU Connected'
    conn.send(msg.encode())
    data = conn.recv(4096)
    from_client = data.decode('utf8')
    print (from_client)
    resp = ValidateRequest(from_client,step,'')
    seed = resp[1]
    conn.send(resp[0].encode())
    if resp[0] == 'End':
        a=1
        b=1
        break
    else: step = 1
    data = conn.recv(4096)
    from_client = data.decode('utf8')
    print (from_client)
    resp = ValidateRequest(from_client,step,seed)
    conn.send(resp[0].encode())
    if resp[0] == 'End':
        a=1
        b=1
        break
    else: 
        print("ECU unlocked")
        a = 1
        b =1
        break
    
  conn.close()
print ('terminating connection')

Tester Connected

27 07 2d2d2d2d2d424547494e205055424c4943204b45592d2d2d2d2d0a4d456b77457759484b6f5a497a6a3043415159494b6f5a497a6a3044415145444d674145314234514b6f764268744d353455624d4a79384b6e3352664e546d300a44552f776b2b53616d48327135793632454756747a69492b692f7670397559722f4e32650a2d2d2d2d2d454e44205055424c4943204b45592d2d2d2d2d0a
67 07 36d52833 2d2d2d2d2d424547494e205055424c4943204b45592d2d2d2d2d0a4d456b77457759484b6f5a497a6a3043415159494b6f5a497a6a3044415145444d6741454a4d517938384275324c5a377342686b446b6c7551495a7442446d780a5172304d6c474572545767486a5467612b6166775943493942657a5556716e66584b30440a2d2d2d2d2d454e44205055424c4943204b45592d2d2d2d2d0a
27 08 081f112b3403b3e3a88b6ad7f6dafa34cea173d1c561c7a786dacada33e9406b7e4350553749098a69d0b71c40dd79c8
<class 'str'>
67 08 
ECU unlocked
terminating connection


In [6]:
randomNo

b'\xe5\x0c\x9f3\xc9\xfb\x93fR\xfct\xcb\xc3\xa5\xaf\x97\xa7\x07\x16\x889\xa8\x17\x8a'

In [11]:
clientPubKey

b'-----BEGIN PUBLIC KEY-----\nMEkwEwYHKoZIzj0CAQYIKoZIzj0DAQEDMgAElAXRRzfn0zU5Ypr/blkQm5STCVe4\n8KVqpPuZEBWN2IktAB2kr0sr+Jn9ku9oCP9M\n-----END PUBLIC KEY-----\n'

In [ ]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import ec
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad, unpad
import socket
import pickle


def generate_ecc_private_key():
    private_key = ec.generate_private_key(ec.SECP192R1(), default_backend())
    return private_key

def generate_ecdh_shared_key(private_key, peer_public_key_bytes):
    peer_public_key = serialization.load_pem_public_key(peer_public_key_bytes, backend=default_backend())
    shared_key = private_key.exchange(ec.ECDH(), peer_public_key)
    return shared_key

serverPriKey = generate_ecc_private_key()

serverPubKeyBytes = serverPriKey.public_key().public_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PublicFormat.SubjectPublicKeyInfo
)

randomNo = ''
clientPubKey = ''
Cryptkey = b'\xb9t5RR\x9d\x1d+\xaa\xa3\xd1}\xfb\x1a\x1d\xcb'


def encrypt_AES(Cryptkey, data):
    data = bytes.fromhex(data)
    cipher = AES.new(Cryptkey, AES.MODE_CBC)
    ciphertext = cipher.encrypt(pad(data, AES.block_size))
    iv = cipher.iv
    return iv + ciphertext

def decrypt_AES(Cryptkey, encrypted_data):
    encrypted_data = bytes.fromhex(encrypted_data)
    iv = encrypted_data[:AES.block_size]
    ciphertext = encrypted_data[AES.block_size:]
    cipher = AES.new(Cryptkey, AES.MODE_CBC, iv)
    plaintext = unpad(cipher.decrypt(ciphertext), AES.block_size)
    return plaintext

def SeedGenerator():
    return get_random_bytes(4).hex()

def ValLevel(lvl):
    if len(lvl)==1:
        return '0'+lvl
    else:
        return lvl
def SeedGen(seed):
    global Cryptkey
    print(type(seed))
    return decrypt_AES(Cryptkey,seed).hex()
def ValidateRequest(from_client, step,key=''):
    secAccLvl = int(from_client[3:5])
    global randomNo, clientPubKey
    if from_client[0:2] == '27' and secAccLvl%2==1:
        if step == 0:
            clientPubKey = bytes.fromhex(from_client[6:len(from_client)])
            randomNo = generate_ecdh_shared_key(serverPriKey,clientPubKey)
            seed = SeedGenerator()
            temp = '67 ' + ValLevel(str(secAccLvl)) + ' '+ seed + ' '+ serverPubKeyBytes.hex()
            print(temp)
            return [temp, seed]
        else : 
            print("Wrong request")
            return ['End','']
    if from_client[0:2] == '27' and secAccLvl%2==0 and key!='':
        if step == 1 and +key == SeedGen(from_client[6:len(from_client)]):
            print('67 '+ ValLevel(str(secAccLvl)),'')
            return ['67 '+ ValLevel(str(secAccLvl)),'']        
        else : 
            print("Wrong request/key not matched/empty")
            return ['End','']

serv = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
serv.bind(('0.0.0.0', 8080))
serv.listen(5)
a=0
b =0
while a==0:
  conn, addr = serv.accept()
  from_client = ''
  step = 0
  while b==0:
    data = conn.recv(4096)
    if not data: break
    from_client = data.decode('utf8')
    print (from_client)
    msg = 'ECU Connected'
    conn.send(msg.encode())
    data = conn.recv(4096)
    from_client = data.decode('utf8')
    print (from_client)
    resp = ValidateRequest(from_client,step,'')
    seed = resp[1]
    conn.send(resp[0].encode())
    if resp[0] == 'End':
        a=1
        b=1
        break
    else: step = 1
    data = conn.recv(4096)
    from_client = data.decode('utf8')
    print (from_client)
    resp = ValidateRequest(from_client,step,seed)
    conn.send(resp[0].encode())
    if resp[0] == 'End':
        a=1
        b=1
        break
    else: 
        print("ECU unlocked")
        a = 1
        b =1
        break
    
  conn.close()
print ('terminating connection')  

In [5]:
import socket
import pickle


for _ in range(0,100):

    serv = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    serv.bind(('0.0.0.0', 8080))
    serv.listen(5)
    a=0
    b =0


    while a==0:
      conn, addr = serv.accept()
      from_client = ''
      step = 0
      while b==0:
        data = conn.recv(4096)
        if not data: break
        from_client = data.decode('utf8')
        print (from_client)
        msg = 'ECU Connected'
        conn.send(msg.encode())
        data = conn.recv(4096)
        from_client = data.decode('utf8')
        print (from_client)
        resp = ValidateRequest(from_client,step,'')
        seed = resp[1]
        conn.send(resp[0].encode())
        if resp[0] == 'End':
            a=1
            b=1
            break
        else: step = 1
        data = conn.recv(4096)
        from_client = data.decode('utf8')
        print (from_client)
        resp = ValidateRequest(from_client,step,seed)
        conn.send(resp[0].encode())
        if resp[0] == 'End':
            a=1
            b=1
            break
        else: 
            print("ECU unlocked")
            a = 1
            b =1
            break

      conn.close()
    print ('terminating connection')

Tester Connected

27 07 2d2d2d2d2d424547494e205055424c4943204b45592d2d2d2d2d0a4d456b77457759484b6f5a497a6a3043415159494b6f5a497a6a3044415145444d6741456f572b4d4831396d766a674c6d664444554732547a5045324d4453610a4c446a46704b41724c374545414b345870564d567a7555776f3370466176515076656d530a2d2d2d2d2d454e44205055424c4943204b45592d2d2d2d2d0a
67 07 de90d582 2d2d2d2d2d424547494e205055424c4943204b45592d2d2d2d2d0a4d456b77457759484b6f5a497a6a3043415159494b6f5a497a6a3044415145444d6741454e5271682f5272796b65424c445579423370485463444673394a43650a6855496b415769744d622f4b6b4f34703932776a39586a39386d47736c4a664d4d4439470a2d2d2d2d2d454e44205055424c4943204b45592d2d2d2d2d0a
27 08 ecd9c9f1d504d937a1aa4211aa1bc972ff19387a33f3094f707b7232266590f949f6c0d6a1b6662ae1f6ffc8812abb9d
<class 'str'>
67 08 
ECU unlocked
terminating connection
Tester Connected

27 07 2d2d2d2d2d424547494e205055424c4943204b45592d2d2d2d2d0a4d456b77457759484b6f5a497a6a3043415159494b6f5a497a6a3044415145444d6741456f572b4d4831396d766a674c6d6644445

27 07 2d2d2d2d2d424547494e205055424c4943204b45592d2d2d2d2d0a4d456b77457759484b6f5a497a6a3043415159494b6f5a497a6a3044415145444d6741456f572b4d4831396d766a674c6d664444554732547a5045324d4453610a4c446a46704b41724c374545414b345870564d567a7555776f3370466176515076656d530a2d2d2d2d2d454e44205055424c4943204b45592d2d2d2d2d0a
67 07 facafba9 2d2d2d2d2d424547494e205055424c4943204b45592d2d2d2d2d0a4d456b77457759484b6f5a497a6a3043415159494b6f5a497a6a3044415145444d6741454e5271682f5272796b65424c445579423370485463444673394a43650a6855496b415769744d622f4b6b4f34703932776a39586a39386d47736c4a664d4d4439470a2d2d2d2d2d454e44205055424c4943204b45592d2d2d2d2d0a
27 08 34f05a2ecdfb5e3713ac4b9e6b255000d3492594ad235fb7e5318621865e7420f5926fe41ba787eae1d490ba7478be68
<class 'str'>
67 08 
ECU unlocked
terminating connection
Tester Connected

27 07 2d2d2d2d2d424547494e205055424c4943204b45592d2d2d2d2d0a4d456b77457759484b6f5a497a6a3043415159494b6f5a497a6a3044415145444d6741456f572b4d4831396d766a674c6d664444554732547a5045324d4

27 07 2d2d2d2d2d424547494e205055424c4943204b45592d2d2d2d2d0a4d456b77457759484b6f5a497a6a3043415159494b6f5a497a6a3044415145444d6741456f572b4d4831396d766a674c6d664444554732547a5045324d4453610a4c446a46704b41724c374545414b345870564d567a7555776f3370466176515076656d530a2d2d2d2d2d454e44205055424c4943204b45592d2d2d2d2d0a
67 07 acd967eb 2d2d2d2d2d424547494e205055424c4943204b45592d2d2d2d2d0a4d456b77457759484b6f5a497a6a3043415159494b6f5a497a6a3044415145444d6741454e5271682f5272796b65424c445579423370485463444673394a43650a6855496b415769744d622f4b6b4f34703932776a39586a39386d47736c4a664d4d4439470a2d2d2d2d2d454e44205055424c4943204b45592d2d2d2d2d0a
27 08 9b04dade6a73d90508621ccd604bba7ec07485ef89c20211fbf9ca705bb9455cda54d992ab422b5b6076bab97504fe64
<class 'str'>
67 08 
ECU unlocked
terminating connection
Tester Connected

27 07 2d2d2d2d2d424547494e205055424c4943204b45592d2d2d2d2d0a4d456b77457759484b6f5a497a6a3043415159494b6f5a497a6a3044415145444d6741456f572b4d4831396d766a674c6d664444554732547a5045324d4

27 07 2d2d2d2d2d424547494e205055424c4943204b45592d2d2d2d2d0a4d456b77457759484b6f5a497a6a3043415159494b6f5a497a6a3044415145444d6741456f572b4d4831396d766a674c6d664444554732547a5045324d4453610a4c446a46704b41724c374545414b345870564d567a7555776f3370466176515076656d530a2d2d2d2d2d454e44205055424c4943204b45592d2d2d2d2d0a
67 07 fe687cdd 2d2d2d2d2d424547494e205055424c4943204b45592d2d2d2d2d0a4d456b77457759484b6f5a497a6a3043415159494b6f5a497a6a3044415145444d6741454e5271682f5272796b65424c445579423370485463444673394a43650a6855496b415769744d622f4b6b4f34703932776a39586a39386d47736c4a664d4d4439470a2d2d2d2d2d454e44205055424c4943204b45592d2d2d2d2d0a
27 08 ca0f1645f5e8a84ad789a8c4f2a283ab17e50f95e36a9603cee84df425df1143c264741a472baacb1550cb6d18453625
<class 'str'>
67 08 
ECU unlocked
terminating connection
Tester Connected

27 07 2d2d2d2d2d424547494e205055424c4943204b45592d2d2d2d2d0a4d456b77457759484b6f5a497a6a3043415159494b6f5a497a6a3044415145444d6741456f572b4d4831396d766a674c6d664444554732547a5045324d4

27 07 2d2d2d2d2d424547494e205055424c4943204b45592d2d2d2d2d0a4d456b77457759484b6f5a497a6a3043415159494b6f5a497a6a3044415145444d6741456f572b4d4831396d766a674c6d664444554732547a5045324d4453610a4c446a46704b41724c374545414b345870564d567a7555776f3370466176515076656d530a2d2d2d2d2d454e44205055424c4943204b45592d2d2d2d2d0a
67 07 d663eb18 2d2d2d2d2d424547494e205055424c4943204b45592d2d2d2d2d0a4d456b77457759484b6f5a497a6a3043415159494b6f5a497a6a3044415145444d6741454e5271682f5272796b65424c445579423370485463444673394a43650a6855496b415769744d622f4b6b4f34703932776a39586a39386d47736c4a664d4d4439470a2d2d2d2d2d454e44205055424c4943204b45592d2d2d2d2d0a
27 08 68c13e23345b58d8e9adff832c878ed5038fe367a3ce2f8266359c567f42d5e76f47111480c29913349a14c8b62bf4a6
<class 'str'>
67 08 
ECU unlocked
terminating connection
Tester Connected

27 07 2d2d2d2d2d424547494e205055424c4943204b45592d2d2d2d2d0a4d456b77457759484b6f5a497a6a3043415159494b6f5a497a6a3044415145444d6741456f572b4d4831396d766a674c6d664444554732547a5045324d4